In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd
from xgboost import XGBRegressor
from scipy.stats.stats import pearsonr   
from sklearn.metrics import mean_absolute_error

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from scipy.stats import skew  # for some statistics
from lightgbm import LGBMRegressor
from scipy.special import boxcox1p
from sklearn.pipeline import make_pipeline
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

X = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
X_test_full = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

#####################
X.drop(['PoolQC'], axis=1, inplace=True)
X.drop(['PoolArea'], axis=1, inplace=True)
X.drop(['Utilities', 'Street'], axis=1, inplace=True)


y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

XConc = pd.concat([X, X_test_full]).reset_index(drop=True)

# XGBoost
my_model_2 = XGBRegressor(eta='0.001', n_estimators=1000, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.55,
                          colsample_bytree=0.2, reg_lambda=3.5, alpha=110000)

objects = []
for i in XConc.columns:
    if XConc[i].dtype == object:
        objects.append(i)
X.update(XConc[objects].fillna('None'))

XConc['LotFrontage'] = X.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
XConc['GarageYrBlt'] = X.groupby('YearBuilt')['GarageYrBlt'].transform(lambda x: x.fillna(x.median())) # -216 sur la MAE

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in XConc.columns:
    if XConc[i].dtype in numeric_dtypes:
        numerics.append(i)
X.update(XConc[numerics].fillna(0))
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in XConc.columns:
    if XConc[i].dtype in numeric_dtypes:
        numerics2.append(i)
skew_features = XConc[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

for i in skew_index:
    XConc[i] = boxcox1p(XConc[i], boxcox_normmax(XConc[i] + 1))

XConc['YrBltAndRemod']=XConc['YearBuilt']+XConc['YearRemodAdd']
XConc['TotalSF']=XConc['TotalBsmtSF'] + XConc['1stFlrSF'] + XConc['2ndFlrSF']

XConc['Total_sqr_footage'] = (XConc['BsmtFinSF1'] + XConc['BsmtFinSF2'] +
                                 XConc['1stFlrSF'] + XConc['2ndFlrSF'])

XConc['Total_Bathrooms'] = (XConc['FullBath'] + (0.5 * XConc['HalfBath']) +
                               XConc['BsmtFullBath'] + (0.5 * XConc['BsmtHalfBath']))

XConc['Total_porch_sf'] = (XConc['OpenPorchSF'] + XConc['3SsnPorch'] +
                              XConc['EnclosedPorch'] + XConc['ScreenPorch'] +
                              XConc['WoodDeckSF'])


final_features = pd.get_dummies(XConc).reset_index(drop=True)
X = final_features.iloc[:len(y), :]
X_sub = final_features.iloc[len(X):, :]

my_model_2.fit(X, y)

print(X[['YearBuilt', 'GarageYrBlt']].corr())

/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:4053: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


             YearBuilt  GarageYrBlt
YearBuilt      1.00000      0.82948
GarageYrBlt    0.82948      1.00000


In [3]:
scores = -1 * cross_val_score(my_model_2, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
15198.788067208903


In [4]:
#obj_cols = [cname for cname in X.columns if
#                    X[cname].dtype == "object"]
#OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
#OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train_full[obj_cols]))
#OH_cols_train.index = X_train_full.index
#num_X_train = X_train_full.drop(obj_cols, axis=1)
#OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
#
#xgb2 = XGBRegressor(n_estimators=800, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.5)
#xgb2.fit(OH_X_train, y_train)





In [5]:
params = {'colsample_bytree': [0.2],
          'lambda': [3.5],
          'learning_rate': [0.005, 0.007, 0.008, 0.01,0.02,0.03,0.04, 0.05],
          'n_estimators' : [100, 200, 300, 400, 500, 600, 800, 900, 1000]
           }

xgb1 = XGBRegressor(n_estimators=800, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.5)

obj_cols = [cname for cname in X.columns if
                    X[cname].dtype == "object"]
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
#OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train_full[obj_cols]))
#OH_cols_train.index = X_train_full.index
#num_X_train = X_train_full.drop(obj_cols, axis=1)
#OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)


gcv = GridSearchCV(xgb1, param_grid=params, cv=2, n_jobs=-1)
#gcv.fit(OH_X_train, y_train)
#print(" Results from Grid Search " )
#print("\n The best estimator across ALL searched params:\n",gcv.best_estimator_)
#print("\n The best score across ALL searched params:\n",gcv.best_score_)
#print("\n The best parameters across ALL searched params:\n",gcv.best_params_)


In [6]:


# Submit
predsSub = my_model_2.predict(X_sub)
output = pd.DataFrame({'Id': X_sub.Id,
                       'SalePrice': predsSub})
output.to_csv('submission.csv', index=False)

